In [ ]:
!pip install tulipy
!pip install pycoingecko
!pip install pandas_ta
!pip install python-binance
!pip install ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tulipy: filename=tulipy-0.4.0-cp310-cp310-linux_x86_64.whl size=288210 sha256=2929bcae06f0dc50ef487e479cdf4d2722c7b20fec1d3491617ef4ca2d8b3cfb
  Stored in directory: /root/.cache/pip/wheels/0a/f1/a7/d402b06d5d6c3fb6270a782341d4753dcd963a00fdde512b07
Successfully built tulipy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=8f5b5dd1df15436572a195bb8f75a05915ce1976e91f0d21ea583e23864b803e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 5.4 MB/s 

# Import of necessary libraries


In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import tulipy as ti
import csv
import builtins
import matplotlib.pyplot as plt
import yfinance as yf
import json
import webbrowser
from datetime import datetime, timedelta
from binance import Client
from jinja2 import Template
import subprocess
import ta
from ta.momentum import RSIIndicator
from ta.trend import ADXIndicator
from ta.trend import SMAIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands

# Data extraction from Binance

In [ ]:
API_Key = "QkGSZFlPppl7a5xc9qyUs8AGBr0gFLDc2XsWlGacTL73SoHf4UHZbk7gTZLNuj9p"
Secret_Key = "gQmP0g7cnzYniHNZjZeIyX5uerTs13wBi16eGyjViFvntDgzjU7LqVOtKs7y7x1P"
client = Client(API_Key,Secret_Key,tld='us')

# Receiving Binance data candles

In [ ]:
def GetData(symbol, period, start, end):
  candles = client.get_historical_klines(symbol, period, start, end)
  return candles

# Database creation

In [ ]:
#Save to csv file
def csvCreate(symbol, candles):
  csvFile = builtins.open(symbol + ".csv", "w", newline='')
  printer = csv.writer(csvFile, delimiter=',')
  for candleData in candles:
    printer.writerow(candleData)
  csvFile.close()

# Retrieve data of entered symbols

In [ ]:
symbolList = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "DOGEUSDT","ATOMUSDT"]
def DataExtractionAndCsvCreation(symbolList):
    for coin in symbolList:
        csvCreate(coin, GetData(coin, Client.KLINE_INTERVAL_1DAY, "19 January 2023", "19 April 2024"))
        print(coin, "csv file created")
DataExtractionAndCsvCreation(symbolList)


BTCUSDT csv file created
ETHUSDT csv file created
BNBUSDT csv file created
DOGEUSDT csv file created
ATOMUSDT csv file created


# Write the Title of the data

In [ ]:
readCsv = "BTCUSDT.csv"
headlines = ['Open_time','Open','High','Low','Close','Volume','Close_time','Quote_asset_volume','Number_of_trades','Taker_buy_base_asset_volume','Taker_buy_quote_asset_volume','Ignore']
df = pd.read_csv(readCsv,names=headlines)
Open=df['Open']
print(Open)

0      20677.13
1      21080.05
2      22675.01
3      22783.64
4      22710.87
         ...   
452    65826.97
453    63380.58
454    63806.00
455    61350.59
456    63447.69
Name: Open, Length: 457, dtype: float64


# Making TimeStamp Data Readable

In [ ]:
def timeCalculate(timestamp):
    return datetime.fromtimestamp(timestamp.timestamp())

def timeWrite():
    for time in Open_time[:5]:
        print(timeCalculate(time))

timeWrite()

2024-12-11 00:00:00
2024-12-11 01:00:00
2024-12-11 02:00:00
2024-12-11 03:00:00
2024-12-11 04:00:00


#Rsi calculation and variable definition

In [ ]:

open = df['Open']
close = df['Close']
Open_time = df['Open_time']
high = df['High']
low = df['Low']

period = 14
rsi_indicator = RSIIndicator(close=close, window=period)
rsi = rsi_indicator.rsi()

print(rsi)

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
452    42.055725
453    43.032891
454    38.629413
455    43.862566
456    44.749538
Name: rsi, Length: 457, dtype: float64


#Moving Medians (sma, adx)


In [ ]:
sma_indicator = SMAIndicator(close, window=50)
sma_50 = sma_indicator.sma_indicator()
print(sma_50)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
452    67007.7032
453    67195.0892
454    67279.5334
455    67300.5316
456    67354.8946
Name: sma_50, Length: 457, dtype: float64


In [ ]:
adx_indicator = ADXIndicator(high, low, close, window=14) #window=period
adx = adx_indicator.adx()

print(adx)

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
         ...    
452    17.825873
453    18.594031
454    19.786650
455    20.894082
456    21.167109
Name: adx, Length: 457, dtype: float64


#Macd Calculation

In [ ]:
macd_indicator = MACD(close, window_slow=26, window_fast=12, window_sign=9)
macd = macd_indicator.macd()

print(macd)


0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
452   -133.090377
453   -387.271583
454   -787.023678
455   -917.819152
456   -979.515786
Name: MACD_12_26, Length: 457, dtype: float64


#Boolinger Band calculation


In [ ]:
bb_indicator = BollingerBands(close, window=20, window_dev=2)
bb = bb_indicator.bollinger_mavg()

print(bb)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
452    68425.7555
453    68146.8350
454    67670.7170
455    67351.3825
456    67055.3975
Name: mavg, Length: 457, dtype: float64


#Buy and sell with Golden and Dead Cross (SMA50-SMA200)

In [ ]:
# Get the data
end = datetime.now().strftime('%Y-%m-%d')
start = datetime.now() - pd.DateOffset(days=30)
data = yf.download("BTC-USD", start=start, end=end, interval="60m")

#Select required data
close = data['Close']
Open_time = data.index
high = data['High']
low = data['Low']

def timeCalculate(timestamp):
    return timestamp.strftime('%Y-%m-%d %H:%M:%S')


def DCA():
    wallet = 100
    purchaseNumber = 0
    saleNumber = 0
    totalCoin = 0
    CommissionRatio = 75 / 10000
    givenCommission = 0

    # Calculate 50 and 200 period simple moving averages
    sma50 = close.rolling(window=50).mean()
    sma200 = close.rolling(window=200).mean()

    print("############################")
    for i in range(1, len(close)):  # i=1'den başlatıyoruz
        if sma50.iloc[i - 1] < sma200.iloc[i - 1] and sma50.iloc[i] > sma200.iloc[i]:
            # Up cross
            if sma50.iloc[i - 1] < sma200.iloc[i - 1] and sma50.iloc[i] > sma200.iloc[i]:
                print(timeCalculate(Open_time[i]), " on the date ", wallet / close.iloc[i], "BTC received")
                purchaseNumber += 1
                totalCoin = wallet / close.iloc[i]
                givenCommission += CommissionRatio * wallet

            # Down cross
            if sma50.iloc[i - 1] > sma200.iloc[i - 1] and sma50.iloc[i] < sma200.iloc[i] and purchaseNumber > 0:
                print(timeCalculate(Open_time[i]), " on the date ", totalCoin, " BTC sold")
                saleNumber += 1
                price = close.iloc[i] * totalCoin
                wallet = price
                totalCoin = 0
                givenCommission += CommissionRatio * price
                print(f"As a result of these two transactions, the transaction value : {wallet}")
                print("#####################################")

    print(f"Total transactions : {(purchaseNumber + saleNumber) / 2}")
    print(f"Total commission paid : {givenCommission}")
    print(f"Total wallet : {wallet}")

DCA()


# Finding the best EMAD and the most profitable crypto based on moving average

In [ ]:
import pandas as pd

def ema(data, period):
    return data.ewm(span=period, adjust=False).mean()

def DCA(symbol, short_period, long_period):
    initial_investment = 50
    buy_count = 0
    total_BTC = 0
    commission_rate = 75 / 10000
    commission = 0

    csv_name = symbol + '.csv'
    attributes = ["Open_time", 'Open', 'High', 'Low', 'Close', 'Volume', '1', '2', '3', '4', '5', '6']
    df = pd.read_csv(csv_name, names=attributes)
    close_prices = df['Close']

    short_ema = ema(df['Close'], period=short_period)
    long_ema = ema(df['Close'], period=long_period)

    for i in range(df.shape[0]):
        if not pd.isna(short_ema[i]) and len(df) > 100 and i > 0:
            if long_ema[i - 1] < short_ema[i - 1] and long_ema[i] > short_ema[i]:  # Buy signal: The long EMA cut the short EMA to the upside.
                buy_count += 1
                total_BTC += initial_investment / close_prices[i]
                commission += commission_rate * initial_investment

    try:
        multiplier = (round(total_BTC, 4) * close_prices[len(close_prices) - 1] - commission) / (initial_investment * buy_count)
    except ZeroDivisionError:
        print(f"{symbol} için hata var")
        return

    total_profit = total_BTC * close_prices[len(close_prices) - 1] - (initial_investment * buy_count) - commission

    '''print("How many times: ”, multiplier)
    print("Total profit: ”, total_profit)
    print("Total Commission: ”, commission)
    print("---------------------------------------------------------------------------------")'''
    return [multiplier, short_period, long_period]

DCA("BTCUSDT", 13, 59)


[2.6313592933333334, 13, 59]

#Code optimization (Best discovery points, Most profitable crypto, Best EMA)

In [ ]:
def Optimization(symbolList):
  rates=[]
  #dcalar = []
  totalrates=[]
  #emaList=[]
  longEmas=[]
  shortEmas=[]
  ema_cuts = []

  for i in symbolList:
    for x in range(2, 21):
      for y in range(50, 61):
        dcaFun = DCA(i, x, y)
        rates.append(dcaFun[0])
        longEmas.append(dcaFun[1])
        shortEmas.append(dcaFun[2])
        ema_cuts.append((dcaFun[1], dcaFun[2] , format(dcaFun[0], ".3f")))

    totalrates.append(max(rates))
    print("################################")
    print(f"coin name : {i}")
    print(f"Best profit : {max(rates)}")
    print(f"The best EMA is the intersection of two : {longEmas[rates.index(max(rates))]} , {shortEmas[rates.index(max(rates))]}")

    #print(f"best win EMA EMA : {rates.index(max(rates))+2}" )
    #emaList.append(rates.index(max(rates))+2)
    print("################################")
    rates.clear()

  print(f"Maximum profit : {max(totalrates)}")
  enCokKazandıranCoin = symbolList[totalrates.index(max(totalrates))]
  print(f"Maximum earning coin : {enCokKazandıranCoin}")
  kesisim_kaydet(ema_cuts)

def kesisim_kaydet(ema_cuts):
    df = pd.DataFrame(ema_cuts, columns=['EMA1', 'EMA2' ,'EMA3'])
    df.to_csv('ema_cuts.csv', index=False)
    print("EMA cuts data successfully saved in 'ema_cuts.csv' file.")

Optimization(['BTCUSDT'])

################################
coin name : BTCUSDT
Best profit : 2.6313592933333334
The best EMA is the intersection of two : 13 , 59
################################
Maximum profit : 2.6313592933333334
Maximum earning coin : BTCUSDT
EMA cuts data successfully saved in 'ema_cuts.csv' file.


# Reading intersection data

In [ ]:
data = pd.read_csv('ema_cuts.csv')
print(data)

     EMA1  EMA2   EMA3
0       2    50  2.210
1       2    51  2.210
2       2    52  2.202
3       2    53  2.202
4       2    54  2.186
..    ...   ...    ...
204    20    56  2.578
205    20    57  2.578
206    20    58  2.578
207    20    59  2.578
208    20    60  2.578

[209 rows x 3 columns]


# Create as many nodes as incoming data and embed the javascript in html

In [ ]:
node = []
Connections = []
for i, row in data.iterrows():
    # Process for each EMA column
    for j, ema_column in enumerate(['EMA1', 'EMA2', 'EMA3']):
        if j == 0 or j == 1:
            radius = min(20, row[ema_column])
        else:
            radius = min(20, (row[ema_column] - row['EMA1']) + (row['EMA2'] - row[ema_column]))
        radius = round(radius, 2)

        # Create and add node data to the list
        dugum = {
            'id': f'node {i * 3 + j + 1}',
            'group': 'Group',
            'radius': radius,
            'citing_patents_count': row[ema_column]
        }
        node.append(dugum)

        # Create links for nodes other than the first node
        if j > 0:
            fark = abs(row[ema_column] - row[data.columns[j - 1]])
            baglanti = {
                'source': f'node {i * 3 + j}',
                'target': f'node {i * 3 + j + 1}',
                'value': round(fark / 10, 4)
            }
            Connections.append(baglanti)

with open("data.json", "w") as data_file:
    json.dump({'nodes': node, 'links': Connections}, data_file)

subprocess.run(["python", "-m", "http.server"])


# **html page **



```
<!-- index.html -->
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>D3.js Graph</title>
  <script src="https://d3js.org/d3.v7.min.js"></script>
  <style>
    svg {
      max-width: 100%;
      height: 100%;
      background-color: black;
    }
    text {
      fill: white;
    }
  </style>
</head>
<body>
  <script>
    const data = {
      nodes: {{ nodes | safe }},
      links: {{ links | safe }}
    };

    async function loadData() {
      const width = window.innerWidth;
      const height = window.innerHeight;
      const color = d3.scaleOrdinal(d3.schemeCategory10);
      const links = data.links.map(function(d) { return Object.assign({}, d); });
      const nodes = data.nodes.map(function(d) { return Object.assign({}, d); });

      const simulation = d3.forceSimulation(nodes)
        .force("link", d3.forceLink(links).id(function(d) { return d.id; }))
        .force("charge", d3.forceManyBody())
        .force("x", d3.forceX())
        .force("y", d3.forceY());

      const svg = d3.create("svg")
        .attr("width", width)
        .attr("height", height)
        .attr("viewBox", [-width / 2, -height / 2, width, height]);

      const link = svg.append("g")
        .attr("stroke", "#f00")
        .attr("stroke-opacity", 0.6)
        .selectAll("line")
        .data(links)
        .join("line")
        .attr("stroke-width", function(d) { return Math.sqrt(d.value); });

      const node = svg.append("g")
        .attr("stroke", "#fff")
        .attr("stroke-width", 1.5)
        .selectAll("g")
        .data(nodes)
        .join("g")
        .attr("transform", d => `translate(${d.x},${d.y})`);

      node.append("circle")
        .attr("r", d => d.radius * 1)
        .attr("fill", "black");

      node.append("text")
        .attr("text-anchor", "middle")
        .attr("dominant-baseline", "middle")
        .attr("font-size", "10px")
        .attr("fill", d => d.group === 'Group' ? 'red' : 'white')
        .text(d => d.citing_patents_count);

      node.call(d3.drag()
        .on("start", dragstarted)
        .on("drag", dragged)
        .on("end", dragended));

      simulation.on("tick", function() {
        link
          .attr("x1", function(d) { return d.source.x; })
          .attr("y1", function(d) { return d.source.y; })
          .attr("x2", function(d) { return d.target.x; })
          .attr("y2", function(d) { return d.target.y; });

        node
          .attr("transform", function(d) { return `translate(${d.x},${d.y})`; });
      });

      function dragstarted(event) {
        if (!event.active) simulation.alphaTarget(0.3).restart();
        event.subject.fx = event.subject.x;
        event.subject.fy = event.subject.y;
      }

      function dragged(event) {
        event.subject.fx = event.x;
        event.subject.fy = event.y;
      }

      function dragended(event) {
        if (!event.active) simulation.alphaTarget(0);
        event.subject.fx = null;
        event.subject.fy = null;
      }

      document.body.appendChild(svg.node());
    }

    window.onload = loadData;
  </script>
</body>
</html>
```



# Generate intersection signals with MACD and calculate profit

In [ ]:
def MACD(symbol, short_period, long_period, signal_period):
    transaction_per_dollar = 50
    buy_number = 0
    total_btc = 0
    commission_rate = 75 / 10000
    Commission = 0

    csv_name = symbol + '.csv'
    columns = ["time",'Open','Max','Min','Close','Volume','1','2','3','4','5','6']
    df = pd.read_csv(csv_name, names=columns)
    Closing = df['Close']

    # Calculate the MACD line
    macd_indicator = ta.trend.macd(df['Close'], window_fast=short_period, window_slow=long_period)
    df['macd_line'] = macd_indicator

    # Calculate signal line
    sinyal_gostergesi = ta.trend.macd_signal(df['Close'], window_fast=short_period, window_slow=long_period, window_sign=signal_period)
    df['signal_line'] = sinyal_gostergesi

    # Find MACD intersections
    crossings = np.where((df['macd_line'].shift(1) < df['signal_line'].shift(1)) & (df['macd_line'] > df['signal_line']), 1, 0)

    # Check intersections and take action
    intersections = []
    for i in range(df.shape[0]):
      if crossings[i] == 1 and len(df) > 100:
        buy_number += 1
        total_btc += transaction_per_dollar / Closing[i]
        Commission += commission_rate * transaction_per_dollar

        '''print(f"Trade value: {df['Close'].iloc[i]}")
        print("Buy/Sell time:", timeCalculate(df['time'].iloc[i]))
        print("Buy/Sell price:", df['Close'].iloc[i])'''

        # Calculate how many times earnings and total earnings
        try:
          multiplier = (np.round(total_btc, 4) * Closing[len(Closing) - 1] - Commission) / (transaction_per_dollar * buy_number)
        except:
          print(f"{symbol}'there's a bug")

        earnings = total_btc * Closing[len(Closing) - 1] - (transaction_per_dollar * buy_number) - Commission

        ''' print("Multiplied by: ", multiplier)
        print("Total earnings: ", earnings)
        print("Toplam verilen Commission: ", Commission)
        print(short_period, long_period, signal_period)
        print("---------------------------------------------------------------------------------")'''

        intersections.append([short_period, long_period, signal_period, multiplier])

    return intersections

def Optimization(symbol_listesi):
    intersections = []

    for symbol in symbol_listesi:
        for short_period in range(10, 21):
            for long_period in range(50, 61):
                for signal_period in range(9, 20):
                    intersections.extend(MACD(symbol, short_period, long_period, signal_period))

    # Let's create a DataFrame over the list of intersections
    columns = ["short_period", "long_period", "signal_period", "multiplier"]
    df = pd.DataFrame(intersections, columns=columns)

    # Let's write it to a CSV file
    df.to_csv('MACD_Section.csv', index=False)
    print("Intersection and earnings data successfully saved in 'intersection_and_earnings.csv' file.")

Optimization(['BTCUSDT'])

Görüntülenen çıkış son 5000 satıra kısaltıldı.
Kaça katladı:  2.2908613200000003
Toplam kazanc:  386.8484245275007
Toplam verilen komisyon:  2.25
14 50 9
---------------------------------------------------------------------------------
Kaça katladı:  2.1996565057142856
Toplam kazanc:  420.81950475562473
Toplam verilen komisyon:  2.625
14 50 9
---------------------------------------------------------------------------------
Kaça katladı:  2.1312528950000003
Toplam kazanc:  450.20883509045734
Toplam verilen komisyon:  3.0
14 50 9
---------------------------------------------------------------------------------
Kaça katladı:  2.03548784
Toplam kazanc:  467.8950396078453
Toplam verilen komisyon:  3.375
14 50 9
---------------------------------------------------------------------------------
Kaça katladı:  1.9844131440000001
Toplam kazanc:  492.453572471695
Toplam verilen komisyon:  3.75
14 50 9
---------------------------------------------------------------------------------
Kaça katladı: 

KeyboardInterrupt: 